In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
   # for filename in filenames:
     #   print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
import numpy as np
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [ ]:
#deleting files to start over
#if os.path.exists('/kaggle/working/train'):
  # shutil.rmtree('/kaggle/working/train')
#if os.path.exists('/kaggle/working/test1'):
   #shutil.rmtree('/kaggle/working/test1')

### run only once

In [ ]:
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
  #  for filename in filenames:
       # print(os.path.join(dirname, filename))


In [ ]:
local_zip = '/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/kaggle/working')
zip_ref.close()

In [ ]:
local_zip ='/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/kaggle/working')
zip_ref.close()

# Counting the images OF CATS / DOGS

In [ ]:
filenames = os.listdir('/kaggle/working/train')
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(str(1))
    else:
        categories.append(str(0))

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

df.head()

In [ ]:
df["category"].value_counts()

### The categories are equally divided into two proportions i.e. there are 12500 pictures of each category. (50%)

In [ ]:
# # Counting the images FOR TRAINING / TESTING

In [ ]:
source_path = '/kaggle/working' 

source_path_train = os.path.join(source_path, 'train')
source_path_test = os.path.join(source_path, 'test')


# os.listdir returns a list containing all files under the given path
print(f"There are {len(os.listdir(source_path_train))} images for training.")
print(f"There are {len(os.listdir(source_path_test))} images for testing.")

# Creating subdirectories for cats and dogs. These will be empty!!  IN A NEW DIRECTORY 'cats-vs-dogs'

In [ ]:
# Define root directory
root_dir = '/kaggle/working/cats-v-dogs' #this is new folder

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
    shutil.rmtree(root_dir)

      
os.makedirs(root_dir + '/training')
os.makedirs(root_dir + '/testing')
os.makedirs(root_dir + '/training/cats')
os.makedirs(root_dir + '/training/dogs')
os.makedirs(root_dir + '/testing/cats')
os.makedirs(root_dir + '/testing/dogs')


In [ ]:
#cheking
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

# Copy the images from the directories train/test1 into 'cats-vs-dogs' which is EMPTY.

In [ ]:
# split_data
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):

    files = []
    for filename in os.listdir(SOURCE):
      file = SOURCE + filename
      if os.path.getsize(file) > 0:
            files.append(filename)
      else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)
    pass



In [ ]:
# Define paths
TRAINING_SOURCE_DIR = "/kaggle/working/train/"
TRAINING_CATS_DIR = "/kaggle/working/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/kaggle/working/cats-v-dogs/testing/cats/"

TEST_SOURCE_DIR = "/kaggle/working/test/"
TRAINING_DOGS_DIR = "/kaggle/working/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/kaggle/working/cats-v-dogs/testing/dogs/"

TRAINING_DIR = "/kaggle/working/cats-v-dogs/training/"
TESTING_DIR = "/kaggle/working/cats-v-dogs/testing/"

# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_CATS_DIR)) > 0:
    for file in os.scandir(TRAINING_CATS_DIR):
        os.remove(file.path)
if len(os.listdir(TRAINING_DOGS_DIR)) > 0:
    for file in os.scandir(TRAINING_DOGS_DIR):
        os.remove(file.path)
if len(os.listdir(TESTING_CATS_DIR)) > 0:
    for file in os.scandir(TESTING_CATS_DIR):
        os.remove(file.path)
if len(os.listdir(TESTING_DOGS_DIR)) > 0:
    for file in os.scandir(TESTING_DOGS_DIR):
        os.remove(file.path)

# Define proportion of images for cats/dogs AS SHOWN BEFORE
split_size = .5

# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(TRAINING_SOURCE_DIR, TRAINING_CATS_DIR, TRAINING_DOGS_DIR, split_size)
split_data(TEST_SOURCE_DIR, TESTING_CATS_DIR, TESTING_DOGS_DIR, split_size)

# Check that the number of images matches the expected output
print(f"\n\nThere are {len(os.listdir(TRAINING_CATS_DIR))} images of cats for training")
print(f"There are {len(os.listdir(TRAINING_DOGS_DIR))} images of dogs for training")
print(f"There are {len(os.listdir(TESTING_CATS_DIR))} images of cats for testing")
print(f"There are {len(os.listdir(TESTING_DOGS_DIR))} images of dogs for testing")

In [ ]:
#train_datagen = ImageDataGenerator(rescale=1/255,
                                    # rotation_range=40,
                                    # width_shift_range=0.2,
                                    # height_shift_range=0.2,
                                    # shear_range=0.2,
                                   #  zoom_range=0.2,
                                   #  horizontal_flip=True,
                                   #  fill_mode='nearest')

train_datagen = ImageDataGenerator(rescale=1/255)
                            
# Pass in the appropriate arguments to the flow_from_directory method
train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=100,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

 # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
validation_datagen =  ImageDataGenerator(rescale=1/255)
    
  # Pass in the appropriate arguments to the flow_from_directory method
validation_generator = validation_datagen.flow_from_directory(directory=TESTING_DIR,
                                                                batch_size=100,
                                                               class_mode='binary',
                                                              #classes=('dogs','cats'),
                                                                target_size=(150, 150))


In [ ]:
from tensorflow.keras.optimizers import RMSprop
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        # Add a dropout rate of 0.2
        tf.keras.layers.Dropout(0.2), 
        tf.keras.layers.Dense(1, activation='sigmoid')
 ])
model.compile(optimizer=RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#class myCallback(tf.keras.callbacks.Callback):
  #def on_epoch_end(self, epoch, logs={}):
   
    # Check accuracy
  #  if(logs.get('accuracy') > 0.9):

      # Stop if threshold is met
      #print("\nAccuracy is higher than 0.9 so cancelling training!")
      #self.model.stop_training = True

# Instantiate class
#callbacks = myCallback()

In [ ]:
# Constant for epochs
EPOCHS = 23


# Train the model
history = model.fit(
      train_generator,
      epochs=EPOCHS,
      validation_data=validation_generator)
     # verbose=2)

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
accuracy=history.history['accuracy']
val_accuracy=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(accuracy)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, accuracy, 'r', "Training Accuracy")
plt.plot(epochs, val_accuracy, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.show()

In [ ]:
model.save_weights('model_catsvsdogs.h5')
model.save('model_catsvsdogs.h5')

# Make categorical prediction:

In [ ]:
validation_generator.reset()
pred= model.predict_generator(validation_generator, verbose=1)

predicted_class_indices=[1 if x >= 0.5 else 0 for x in pred]

labels=(validation_generator.class_indices)
labels2=dict((v,k) for k,v in labels.items())
predictions=[labels2[k] for k in predicted_class_indices]

print(predicted_class_indices)
print(labels)
print(predictions)

# Generate .csv for submission

In [ ]:
counter = range(1, len(predictions) + 1)
solution = pd.DataFrame({"id": counter, "label":predicted_class_indices})
solution.to_csv("dogsVScats.csv", index = False)
